# make donor

In [1]:
import os.path as op

import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
#from mne.datasets import somato
import copy

import os

In [2]:
subj = 'P001'
r = 2
cond = 'risk'
fb = 'negative'

L_freq = 2
H_freq = 40
f_step = 1
freqs = np.arange(L_freq, H_freq, f_step)
#for others
n_cycles = freqs//2

period_start = -1.750
period_end = 2.750

baseline = (-0.35, -0.05)

data_path = '/net/server/data/Archive/prob_learn/vtretyakova/ICA_cleaned'

In [8]:
events_pos = np.loadtxt("/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/fix_cross_mio_corr/{0}_run{1}_norisk_fb_cur_positive_fix_cross.txt".format(subj, r), dtype='int') 
if events_pos.shape == (3,):
    events_pos = events_pos.reshape(1,3)
    
events_neg = np.loadtxt("/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/fix_cross_mio_corr/{0}_run{1}_norisk_fb_cur_negative_fix_cross.txt".format(subj, r), dtype='int')
if events_neg.shape == (3,):
    events_neg = events_neg.reshape(1,3) 
    
#объединяем негативные и позитивные фидбеки для получения общего бейзлайна по ним, и сортируем массив, чтобы времена меток шли в порядке возрастания    
events = np.vstack([events_pos, events_neg])
events = np.sort(events, axis = 0) 

#events, which we need
events_response = np.loadtxt('/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/events_by_cond_mio_corrected/{0}_run{1}_{2}_fb_cur_{3}.txt'.format(subj, r, cond, fb), dtype='int')
# если только одна метка, т.е. одна эпоха, то выдается ошибка, поэтому приводи shape к виду (N,3)
if events_response.shape == (3,):
    events_response = events_response.reshape(1,3)
    
    
raw_fname = op.join(data_path, '{0}/run{1}_{0}_raw_ica.fif'.format(subj, r))

raw_data = mne.io.Raw(raw_fname, preload=True)
        
    
picks = mne.pick_types(raw_data.info, meg = 'planar1', eog = True)

#epochs for baseline
    # baseline = None, чтобы не вычитался дефолтный бейзлайн
epochs = mne.Epochs(raw_data, events, event_id = None, tmin = -1.0, tmax = 1.0, baseline = None, picks = picks, preload = True)
epochs.resample(300)

freq_show_baseline = mne.time_frequency.tfr_multitaper(epochs, freqs = freqs, n_cycles = n_cycles, use_fft = False, return_itc = False, average=False).crop(tmin=baseline[0], tmax=baseline[1], include_tmax=True) #frequency of baseline

b_line = np.swapaxes(freq_show_baseline.data, 0, 1)
b_line = np.swapaxes(b_line, 1, 2)
b_line.shape

a, b, c, d = b_line.shape
b_line = b_line.reshape(a, b, c * d)
b_line.shape

####### ДЛЯ ДАННЫХ ##############
# baseline = None, чтобы не вычитался дефолтный бейзлайн
epochs = mne.Epochs(raw_data, events_response, event_id = None, tmin = period_start, 
		                tmax = period_end, baseline = None, picks = picks, preload = True)
		       
epochs.resample(300) 

freq_show = mne.time_frequency.tfr_multitaper(epochs, freqs = freqs, n_cycles = n_cycles, use_fft = False, return_itc = False, average=False)

####### Для данных так же меняем оси местами
data = np.swapaxes(freq_show.data, 0, 1)
#data.shape
data = np.swapaxes(data, 1, 2)
data = np.swapaxes(data, 2, 3)
# Усредняем бейзлайн по всем точкам, получаем одно число (которое будем вычитать из data для каждого канала)
	    
b = b_line.mean(axis=-1)
	    
    
b_line_new_shape = b[:, :, np.newaxis, np.newaxis]
#b_line_new_shape.shape

data = 10*np.log10(data/b_line_new_shape)

data = np.swapaxes(data, 2, 3)
data = np.swapaxes(data, 1, 2)
data = np.swapaxes(data, 0, 1)
data.shape

Opening raw data file /net/server/data/Archive/prob_learn/vtretyakova/ICA_cleaned/P001/run2_P001_raw_ica.fif...
    Range : 154000 ... 303999 =    154.000 ...   303.999 secs
Ready.
Reading 0 ... 149999  =      0.000 ...   149.999 secs...


<ipython-input-8-3ffef112c9de>:22: RuntimeWarning: This filename (/net/server/data/Archive/prob_learn/vtretyakova/ICA_cleaned/P001/run2_P001_raw_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_data = mne.io.Raw(raw_fname, preload=True)


Not setting metadata
Not setting metadata
25 matching events found
No baseline correction applied
0 projection items activated
Loading data for 25 events and 2001 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Loading data for 2 events and 4501 original time points ...
0 bad epochs dropped
Not setting metadata


(2, 102, 38, 1350)

In [9]:
freq_show.data = data
freq_show.save('/home/vtretyakova/Рабочий стол/time_frequency_plots/donor_combined_planars.h5', overwrite=True)

In [ ]:
# 